In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from data import fut_list, fut_read, stock_read
from util import adf_test, data_generator, significant, ohansen_cointegration_test

# Config

In [2]:
feature = 'close'

# DATA
* 数据基础已经全部存在data里了。只需进一步处理得到想要的时间序列。

In [3]:
fut = 'CU9999.XSGE'
fut_df = fut_read(fut)
stock_df = stock_read(fut)

/Users/hejizhang/Downloads/主连日线-2018年后/data.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fut['ChangeRatio'].iloc[0] = 0


### 如果想要定制特征，处理fut_df \ stock_df即可。

# ADF验证是否平稳

In [5]:
adf_test(fut_df[feature])
for idx, df in enumerate(stock_df):
    adf_test(df[feature])
    

Test Statistic                   -1.247847
p-value                           0.652682
#Lags Used                        2.000000
Number of Observations Used    1191.000000
Critical Value (1%)              -3.435852
Critical Value (5%)              -2.863970
Critical Value (10%)             -2.568064
dtype: float64
Test Statistic                   -0.678746
p-value                           0.852178
#Lags Used                        5.000000
Number of Observations Used    1184.000000
Critical Value (1%)              -3.435885
Critical Value (5%)              -2.863984
Critical Value (10%)             -2.568071
dtype: float64
Test Statistic                   -1.854404
p-value                           0.353805
#Lags Used                       13.000000
Number of Observations Used    1180.000000
Critical Value (1%)              -3.435904
Critical Value (5%)              -2.863992
Critical Value (10%)             -2.568076
dtype: float64
Test Statistic                   -1.507070
p-value  

# 根据模型找出各个stock的联动显著时间段
* 由于只有VAR，故不再对模型选择进行分支。

In [3]:
sig_col = []
for idx, df in enumerate(stock_df):
    data = data_generator(fut_df, df, feature)
    sig_col.append(significant(data))
    

sig_col 即为显著的index

还有更进一步的算最佳lead lag,不过都是1，所以似乎不重要了,写在下面。可以尝试下冲击

```python
lag_order = model.select_order(15)
print(f"Selected lag order: {lag_order.selected_orders['aic']}")
model_fitted = model.fit(maxlags=lag_order.selected_orders['aic'])
print(model_fitted.summary())

irf = model_fitted.irf(10) # 10期冲击响应
irf.plot(orth=True) # 正交化冲击响应图
plt.show()

```

# 协整检验
* 只用检验上面的data数据（就是一列是期货数据，一列是股票数据）

In [ ]:

for idx, df in enumerate(stock_df):
    data = data_generator(fut_df, df, feature)
    